In [3]:
import nltk
import codecs#to convert utf-8 to latin2
import random
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC,LinearSVC,NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize

In [15]:
class VoteClassifier(ClassifierI):
    def __init__(self,*classifiers):
        self._classifiers=classifiers
    
    def classify(self,features):
        votes=[]
        for c in self._classifiers:
            v=c.classify(features)
            votes.append(v)
        
        #print(votes)
        return mode(votes)
    def confidence(self,features):
        votes=[]
        for c in self._classifiers:
            v=c.classify(features)
            votes.append(v)
            
        choice_votes=votes.count(mode(votes))
        print(choice_votes)
        conf=choice_votes/len(votes)
        
        return conf
short_pos = codecs.open("positive.txt","r", encoding='latin2').read()
short_neg = codecs.open("negative.txt","r", encoding='latin2').read()
#short_pos=open("positive.txt","r").read()
#short_neg=open("negative.txt","r").read()

documents=[]

for review in short_pos.split('\n'):
    documents.append((review,"pos"))
for review in short_pos.split('\n'):
    documents.append((review,"neg"))

all_words=[]

short_words_pos=word_tokenize(short_pos)
short_words_neg=word_tokenize(short_neg)

for w in short_words_pos:
    all_words.append(w.lower())
    
for w in short_words_neg:
    all_words.append(w.lower())


all_words=nltk.FreqDist(all_words)

word_features=list(all_words.keys())[:5000]

def find_features(document):
    words=word_tokenize(document)
    features={}
    for w in word_features:
        features[w]= (w in words)
    
    return features
    
featuresets=[(find_features(rev),category) for (rev,category) in documents]

random.shuffle(featuresets)

training_set=featuresets[:10000]
test_set=featuresets[10000:]


In [5]:
#NAIVE BAYES 

classifier=nltk.NaiveBayesClassifier.train(training_set)


print("original_accuracy by naive_bayes:",nltk.classify.accuracy(classifier,test_set)*100)

classifier.show_most_informative_features(15)

original_accuracy by naive_bayes: 14.006024096385541
Most Informative Features
                 passing = True              pos : neg    =      2.3 : 1.0
              transition = True              pos : neg    =      2.3 : 1.0
             desperately = True              pos : neg    =      2.3 : 1.0
                historia = True              pos : neg    =      2.3 : 1.0
               palatable = True              neg : pos    =      2.3 : 1.0
                  cletis = True              neg : pos    =      2.3 : 1.0
                graceful = True              neg : pos    =      2.3 : 1.0
                   dumas = True              neg : pos    =      2.2 : 1.0
                    sick = True              pos : neg    =      2.1 : 1.0
                 brosnan = True              neg : pos    =      1.9 : 1.0
                  hidden = True              pos : neg    =      1.8 : 1.0
                absolute = True              pos : neg    =      1.8 : 1.0
                     

In [6]:
#MNB CLASSIFIER

MNB_classifier=SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("mnb_accuracy:",nltk.classify.accuracy(MNB_classifier,test_set)*100)

mnb_accuracy: 14.006024096385541


In [7]:
#USING bernoulliCLASSSIFIER

BN_classifier=SklearnClassifier(BernoulliNB())
BN_classifier.train(training_set)
print("bn_accuracy:",nltk.classify.accuracy(BN_classifier,test_set)*100)

bn_accuracy: 14.307228915662652


In [8]:
#LOGISTIC REGRESSION

logistic_regression=SklearnClassifier(LogisticRegression())
logistic_regression.train(training_set)
print("logistic_regression accuracy:",nltk.classify.accuracy(logistic_regression,test_set)*100)

/home/chiranjeev/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


logistic_regression accuracy: 13.102409638554215


In [9]:
#SGD CLASSIFIER
SGDClassifier=SklearnClassifier(SGDClassifier())
SGDClassifier.train(training_set)
print("SGDClassifier accuracy:",nltk.classify.accuracy(SGDClassifier,test_set)*100)

/home/chiranjeev/.local/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier accuracy: 29.819277108433734


In [10]:
#SVC CLASSIFIER
SVC_Classifier=SklearnClassifier(SVC())
SVC_Classifier.train(training_set)
print("SVC_Classifier accuracy:",nltk.classify.accuracy(SVC_Classifier,test_set)*100)

/home/chiranjeev/.local/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC_Classifier accuracy: 49.39759036144578


In [11]:
#LINEAR CLASSIFIER

LinearSVC_Classifier=SklearnClassifier(LinearSVC())
LinearSVC_Classifier.train(training_set)
print("LinearSVC_Classifier accuracy:",nltk.classify.accuracy(LinearSVC_Classifier,test_set)*100)

LinearSVC_Classifier accuracy: 12.198795180722891


In [12]:
#NUSVC CLASSIFIER
NuSVC_Classifier=SklearnClassifier(NuSVC())
NuSVC_Classifier.train(training_set)
print("NuSVC_Classifier accuracy:",nltk.classify.accuracy(NuSVC_Classifier,test_set)*100)

NuSVC_Classifier accuracy: 60.8433734939759


In [17]:
voted_classifier=VoteClassifier(MNB_classifier,BN_classifier,logistic_regression,LinearSVC_Classifier,NuSVC_Classifier)#,SGDClassifier,classifier,SVC_Classifier)

print("voted_classifier accuracy:",nltk.classify.accuracy(voted_classifier,test_set)*100)
print("classfication:",voted_classifier.classify(test_set[0][0]),voted_classifier.confidence(test_set[0][0])*100)
print("classfication:",voted_classifier.classify(test_set[1][0]),voted_classifier.confidence(test_set[1][0])*100)
print("classfication:",voted_classifier.classify(test_set[4][0]),voted_classifier.confidence(test_set[4][0])*100)
print("classfication:",voted_classifier.classify(test_set[3][0]),voted_classifier.confidence(test_set[3][0])*100)

voted_classifier accuracy: 48.795180722891565
5
classfication: neg 100.0
5
classfication: pos 100.0
4
classfication: pos 80.0
5
classfication: neg 100.0
